## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

In [0]:
import pandas as pd
import numpy as np
import keras
import zipfile as zf
%matplotlib inline
import os

import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
from glob import glob
import seaborn as sns

from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

### Load Dataset Files

In [70]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [71]:
%cd /gdrive/'My Drive/Data/CNN2'

/gdrive/My Drive/Data/CNN2


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/gdrive/'My Drive/Data/CNN2"

In [0]:
file = zf.ZipFile('train.zip')
file.extractall()


Repeat the same step for test.zip

In [0]:
file1 = zf.ZipFile('test.zip')
file1.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
file = zf.ZipFile('sample_submission.csv.zip')
file.extractall()

Repeat the same step for labels.csv.zip

In [0]:
file = zf.ZipFile('labels.csv.zip')
file.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
labels = pd.read_csv("labels.csv")

### Print the count of each category of Dogs given in the dataset



In [0]:
breed_counts = labels['breed'].value_counts()

In [7]:
breed_counts.head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

In [8]:
breed_counts.shape

(120,)

### Get one-hot encodings of labels

In [0]:
target = pd.Series(labels['breed'])
one_hot = pd.get_dummies(target,sparse=True)
target = np.asarray(one_hot)

In [16]:
target[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
im_size =100
num_channels =3

In [0]:
y_train=target

In [0]:
im_size=100
ids=labels['id']
X_train=[]

for id in ids:
    img=cv2.imread('./train/{}'.format(id)+'.jpg')
    X_train.append(cv2.resize(img,(im_size,im_size)))
X_train=np.array(X_train,np.float32)

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
def standardize(array):
    array/=255
    return array


In [0]:
X_train=standardize(X_train)


### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=7)

In [0]:
X_test, X_val, y_test, Y_val = train_test_split(X_test,y_test, test_size = 0.5, random_state=7)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_img=pd.read_csv("./sample_submission.csv")

In [0]:
x_test_feature=list()
ids_test=test_img['id']
for id in ids_test:
    img=cv2.imread('./test/{}'.format(id)+'.jpg')
    x_test_feature.append(cv2.resize(img,(im_size,im_size)))
x_test_feature=np.array(x_test_feature,np.float32)

Run the below code to load the test image files in x_test_feature

Normalize the test data and convert it into 4 dimensions

In [0]:
X_test=standardize(x_test_feature)

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
num_classes=120
batch_size=128
epochs=10

In [0]:
model=Sequential()
model.add(Conv2D(32,(5,5),input_shape=X_train.shape[1:],activation='relu',padding='same'))
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 100, 100, 32)      2432      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
flatten_2 (Flatten)          (None, 640000)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               163840256 
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 120)               30840     
_________________________________________________________________
activation_6 (Activation)    (None, 120)               0         
Total para

### Use batch_size = 128 and epochs = 10 and execute the model

In [35]:
model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)

Epoch 1/10
7155/7155 [==============================] - 19s 3ms/step - loss: 4.7874 - acc: 0.0084
Epoch 2/10
7155/7155 [==============================] - 13s 2ms/step - loss: 4.7847 - acc: 0.0124
Epoch 3/10
7155/7155 [==============================] - 13s 2ms/step - loss: 4.7819 - acc: 0.0123
Epoch 4/10
7155/7155 [==============================] - 13s 2ms/step - loss: 4.7788 - acc: 0.0127
Epoch 5/10
7155/7155 [==============================] - 14s 2ms/step - loss: 4.7768 - acc: 0.0127
Epoch 6/10
7155/7155 [==============================] - 14s 2ms/step - loss: 4.7755 - acc: 0.0127
Epoch 7/10
7155/7155 [==============================] - 14s 2ms/step - loss: 4.7750 - acc: 0.0127
Epoch 8/10
7155/7155 [==============================] - 13s 2ms/step - loss: 4.7749 - acc: 0.0127
Epoch 9/10
7155/7155 [==============================] - 13s 2ms/step - loss: 4.7747 - acc: 0.0127
Epoch 10/10
7155/7155 [==============================] - 13s 2ms/step - loss: 4.7746 - acc: 0.0127


#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False, 
    samplewise_center=False, 
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False)

In [0]:
datagen.fit(X_train)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [44]:
output = model.fit_generator(datagen.flow(X_train, y_train, batch_size=32), 
                           use_multiprocessing=True,
                     steps_per_epoch=len(X_test) / 32, epochs=1)

Epoch 1/1
324/323 [==============================] - 47s 146ms/step - loss: 4.7755 - acc: 0.0124


### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'), pooling='avg')

Print the summary of the base_model

In [0]:
base_model.summary()

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
base_model.add(Flatten())
base_model.add(Dense(1024))
base_model.add(Activation('relu'))
base_model.add(Dense(512))
base_model.add(Activation('relu'))
base_model.add(Dense(256))
base_model.add(Activation('relu'))
base_model.add(Dense(num_classes))
base_model.add(Activation('softmax'))
base_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:

for layer in base_model.layers:
  layer.trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
base_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)